In [4]:
# import dependencies
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
from splinter import Browser
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()

In [35]:
# get draft data
engine = create_engine('sqlite:///../data/fbdata.sqlite')
Base.metadata.create_all(engine)
Base.prepare(engine, reflect=True)
session = Session(engine)
draftDf = pd.read_sql_query("SELECT Player, CollegeStats FROM draftHistory WHERE Pos = 'QB'", con=engine)
draftDf

,DraftYear,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,...,RushingYds,RushingTD,ReceivingRec,ReceivingYds,ReceivingTD,ReceivingSolo,Int,Sk,College/Univ,CollegeStats
0,1999,1,1,CLE,Tim Couch,QB,22,2003,0,0,...,556,2,2,-13,0,None,None,None,Kentucky,http://www.sports-reference.com/cfb/players/ti...
1,1999,1,2,PHI,Donovan McNabb,QB,22,2011,0,6,...,3459,29,5,-7,0,None,None,None,Syracuse,http://www.sports-reference.com/cfb/players/do...
2,1999,1,3,CIN,Akili Smith,QB,24,2002,0,0,...,371,1,1,6,0,None,None,None,Oregon,http://www.sports-reference.com/cfb/players/ak...
3,1999,1,11,MIN,Daunte Culpepper,QB,22,2009,0,3,...,2652,34,1,0,0,None,None,None,Central Florida,http://www.sports-reference.com/cfb/players/da...
4,1999,1,12,CHI,Cade McNown,QB,22,2000,0,0,...,486,3,0,0,0,None,None,None,UCLA,http://www.sports-reference.com/cfb/players/ca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2020,6,189,JAX,Jake Luton,QB,None,None,0,0,...,None,None,None,None,None,None,None,None,Oregon St.,http://www.sports-reference.com/cfb/players/ja...
271,2020,7,224,TEN,Cole McDonald,QB,None,None,0,0,...,None,None,None,None,None,None,None,None,Hawaii,http://www.sports-reference.com/cfb/players/co...
272,2020,7,231,DAL,Ben DiNucci,QB,None,None,0,0,...,None,None,None,None,None,None,None,None,James Madison,http://www.sports-reference.com/cfb/players/be...
273,2020,7,240,NOR,Tommy Stevens,QB,23,None,0,0,...,None,None,None,None,None,None,None,None,Mississippi St.,http://www.sports-reference.com/cfb/players/to...


In [36]:
# loop through links.
cfbStatsDf = pd.DataFrame(columns = ['year','school','conf','class','pos','games','rushAtt','rushYds','rushAvg','rushTD','rec',\
                               'recYds','recAvg','recTD','plays','ydsFromScrim','ydsFromScrimAvg','rushRecTD'])
for index, row in draftDf.iterrows():
    if row.CollegeStats:
        print(row.CollegeStats)
        soup = bs(requests.get(row.CollegeStats).text, 'lxml')
        try:
            tables = soup.findAll(class_='table_wrapper')
            all_passing = pd.read_html(soup.find(id=f"all_passing").prettify(), header=1)[0]
            all_passing.columns = ['year','school','conf','class','pos','games','passComp','passAtt','compPct','passYds',\
                                   'ydsPerAtt','adjYdsPerAtt','TD','int','effRate']
            all_passing.year = all_passing.year.str.strip('*').str.strip(' ')
            all_rushing = pd.read_html(soup.find(id=f"all_rushing").findAll(text=lambda text:isinstance(text, Comment))[0].extract(), header=1)[0]
            all_rushing.columns = ['year','school','conf','class','pos','games','rushAtt','rushYds','rushAvg','rushTD','rec',\
                                   'recYds','recAvg','recTD','plays','ydsFromScrim','ydsFromScrimAvg','rushRecTD']
            all_rushing.year = all_passing.year.str.strip('*').str.strip(' ')
            # all_passing.head()
            playerdf = pd.merge(all_passing, all_rushing, on=['year','school','conf','class','pos','games'])
            playerdf['name'] = row.Player
            cfbStatsDf = cfbStatsDf.append(playerdf, ignore_index=True)
        except:
            print('fail')
        

http://www.sports-reference.com/cfb/players/tim-couch-1.html
http://www.sports-reference.com/cfb/players/donovan-mcnabb-1.html
http://www.sports-reference.com/cfb/players/akili-smith-1.html
http://www.sports-reference.com/cfb/players/daunte-culpepper-1.html
http://www.sports-reference.com/cfb/players/cade-mcnown-1.html
http://www.sports-reference.com/cfb/players/shaun-king-1.html
http://www.sports-reference.com/cfb/players/brock-huard-1.html
http://www.sports-reference.com/cfb/players/joe-germaine-1.html
http://www.sports-reference.com/cfb/players/aaron-brooks-1.html
http://www.sports-reference.com/cfb/players/michael-bishop-1.html
http://www.sports-reference.com/cfb/players/scott-covington-1.html
http://www.sports-reference.com/cfb/players/chad-pennington-1.html
http://www.sports-reference.com/cfb/players/chris-redman-1.html
http://www.sports-reference.com/cfb/players/tee-martin-1.html
http://www.sports-reference.com/cfb/players/marc-bulger-1.html
http://www.sports-reference.com/cfb/p

http://www.sports-reference.com/cfb/players/jimmy-clausen-1.html
http://www.sports-reference.com/cfb/players/colt-mccoy-1.html
http://www.sports-reference.com/cfb/players/mike-kafka-1.html
http://www.sports-reference.com/cfb/players/rusty-smith-1.html
http://www.sports-reference.com/cfb/players/tony-pike-1.html
http://www.sports-reference.com/cfb/players/levi-brown-1.html
http://www.sports-reference.com/cfb/players/cam-newton-1.html
http://www.sports-reference.com/cfb/players/jake-locker-1.html
http://www.sports-reference.com/cfb/players/blaine-gabbert-1.html
http://www.sports-reference.com/cfb/players/christian-ponder-1.html
http://www.sports-reference.com/cfb/players/andy-dalton-1.html
http://www.sports-reference.com/cfb/players/colin-kaepernick-1.html
http://www.sports-reference.com/cfb/players/ryan-mallett-1.html
http://www.sports-reference.com/cfb/players/ricky-stanzi-1.html
http://www.sports-reference.com/cfb/players/tj-yates-1.html
http://www.sports-reference.com/cfb/players/nat

In [37]:
# get player data
cfbStatsDf


,year,school,conf,class,pos,games,rushAtt,rushYds,rushAvg,rushTD,...,passComp,passAtt,compPct,passYds,ydsPerAtt,adjYdsPerAtt,TD,int,effRate,name
0,1996,Kentucky,SEC,NaN,QB,7.0,24,-26,-1.1,0,...,32.0,84.0,38.1,276.0,3.3,3.0,1.0,1.0,67.2,Tim Couch
1,1997,Kentucky,SEC,NaN,QB,11.0,66,-125,-1.9,3,...,363.0,547.0,66.4,3884.0,7.1,6.9,37.0,19.0,141.4,Tim Couch
2,1998,Kentucky,SEC,NaN,QB,11.0,64,-124,-1.9,1,...,400.0,553.0,72.3,4275.0,7.7,7.8,36.0,15.0,153.3,Tim Couch
3,Career,Kentucky,NaN,NaN,NaN,NaN,154,-275,-1.8,4,...,795.0,1184.0,67.1,8435.0,7.1,7.0,74.0,35.0,141.7,Tim Couch
4,1995,Syracuse,Big East,NaN,QB,11.0,123,261,2.1,2,...,128.0,207.0,61.8,1991.0,9.6,9.9,16.0,6.0,162.3,Donovan McNabb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,2016,Iowa,Big Ten,FR,QB,5.0,0,0,NaN,0,...,5.0,9.0,55.6,62.0,6.9,6.9,0.0,0.0,113.4,Nate Stanley
1185,2017,Iowa,Big Ten,SO,QB,13.0,49,-115,-2.3,0,...,196.0,351.0,55.8,2432.0,6.9,7.6,26.0,6.0,135.1,Nate Stanley
1186,2018,Iowa,Big Ten,JR,QB,13.0,36,4,0.1,1,...,235.0,396.0,59.3,2852.0,7.2,7.4,26.0,10.0,136.5,Nate Stanley
1187,2019,Iowa,Big Ten,SR,QB,13.0,76,7,0.1,1,...,237.0,399.0,59.4,2951.0,7.4,7.4,16.0,7.0,131.2,Nate Stanley


In [38]:
# commit to sqlite database
cfbStatsDf.to_sql('CFBStats', con=engine, if_exists='replace', index=False)